# Data Engineering Python Test

In [1]:
# Run this cell to delete existent parquet files and re-produce the conversion.
!rm -rf pyarrow/basic/
!rm -rf fastparquet/basic/

In [2]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq 
import fastparquet as fsp 
import glob

In [3]:
df201 = pd.read_csv('weather.20160201.csv', parse_dates=["ObservationDate"])
df301 = pd.read_csv("weather.20160301.csv", parse_dates=['ObservationDate'])

In [4]:
print(df201.shape, df301.shape)

(93255, 15) (101442, 15)


## Parquet supported packages
There is two widely used package within the python ecosystem which is pyarrow and fastparquest. Pyarrow is c++ implementation of the Parquet file format that allows efficient implementation in python. On the other hand fastparquet is python based implementation of the Parquet file takes advantage of the numbas JIT compiler. Despite the fact pandas library allows conversion between file formats to Parquet using either of these packages, more granular customization for the I/O operations can be achieve using the packages explicitly. 

## Pyarrow conversion

In [5]:
tbl201 = pa.Table.from_pandas(df201, preserve_index=False)
tbl301 = pa.Table.from_pandas(df301, preserve_index=False)

In [6]:
pq.write_to_dataset(tbl201, root_path="pyarrow/basic")
pq.write_to_dataset(tbl301, root_path="pyarrow/basic")

In [7]:
%%timeit
#Only read the necesary columns
pyarrow_tbl = pq.ParquetDataset("pyarrow/basic")
full_df = pyarrow_tbl.read(columns=["ObservationDate",
"ScreenTemperature",
"Region"]).to_pandas()

37.2 ms ± 6.22 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
pyarrow_tbl = pq.ParquetDataset("pyarrow/basic")
full_df = pyarrow_tbl.read(columns=["ObservationDate",
"ScreenTemperature",
"Region"]).to_pandas()

In [22]:
len(full_df) == len(df201) + len(df301)

True

In [23]:
full_df.columns

Index(['ObservationDate', 'ScreenTemperature', 'Region'], dtype='object')

In [24]:
full_df.loc[full_df["ScreenTemperature"] == full_df["ScreenTemperature"].max(), ["ObservationDate"]]

,ObservationDate
147768,2016-03-17


In [25]:
full_df.loc[full_df["ScreenTemperature"] == full_df["ScreenTemperature"].max(), ["ObservationDate", "ScreenTemperature"]]

,ObservationDate,ScreenTemperature
147768,2016-03-17,15.8


In [26]:
full_df.loc[full_df["ScreenTemperature"] == full_df["ScreenTemperature"].max(), ["Region"]]

,Region
147768,Highland & Eilean Siar


## Fastparquet 

In [27]:
fsp.write(filename="fastparquet/basic", data=df201, file_scheme='hive', write_index=False)

In [28]:
fsp.write(filename="fastparquet/basic/", data=df301, file_scheme="hive", write_index=False, append=True)

In [29]:
mdata_max = fsp.ParquetFile("fastparquet/basic/_metadata")

In [30]:
max_temp_list = mdata_max.statistics['max']['ScreenTemperature']


In [31]:
max_index = max_temp_list.index(max(max_temp_list))

Read only the relevant file and relevant columns wilth the index

In [32]:
%%timeit
related_file = fsp.ParquetFile(glob.glob("fastparquet/basic/*")[max_index]).to_pandas(columns=['ObservationDate', 'ScreenTemperature', 'Region'])

72.3 ms ± 12.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
related_file = fsp.ParquetFile(glob.glob("fastparquet/basic/*")[max_index]).to_pandas(columns=['ObservationDate', 'ScreenTemperature', 'Region'])

In [35]:
related_file.loc[related_file['ScreenTemperature'] == related_file['ScreenTemperature'].max()]

,ObservationDate,ScreenTemperature,Region
147768,2016-03-17,15.8,Highland & Eilean Siar


## Optimization related issues

Both of these libraries allow finer grained implementations such as writing files in desired row groups based on the number of rows and writing partitioned files based on the partition columns. 
For the files we have it wouldn't be useful to have any partitioned implementations because the file sizes are relatively small. 

### Avoiding a lot of small files
It is advisable to avoiding to write too many small files because every parquet file includes metadata in the footer of the file. Hence having too many small files lead to writing similar metadata over and over again and using more storage than needed.

### Avoiding very large files
It is also advisable to avoid have a very large file even though its a high dimensional data and file needs to be read only selected columns. Because of the very large file can have very long footer metadata because of the record of the unique values stored in the metadata.